In [1]:
#Préparation pour le KNN
#Suppression des deux dernières colonnes
df_toAnalyse = df_clean.drop(["movie_title",'country'],axis=1).to_numpy()

# Centrage et Réductions afin de ne pas le répéter dans les différents algorithme
std_scale = preprocessing.StandardScaler().fit(df_toAnalyse)
df_CentreReduit = std_scale.transform(df_toAnalyse)

NameError: name 'df_clean' is not defined

In [ ]:
#Algorithme KNN Plus proche voisin de façon BRUT sans ACP
nbrs = NearestNeighbors(n_neighbors=6, algorithm='ball_tree').fit(df_CentreReduit)
dist_KNN_BRUTE, ind_KNN_BRUTE = nbrs.kneighbors(df_CentreReduit)

In [ ]:
#KNN après ACP
# choix du nombre de composantes à calculer
n_comp = 8

# Calcul des composantes principales
pca = decomposition.PCA(n_components=n_comp)
pca.fit(df_CentreReduit)
X_ACP = pca.transform(df_CentreReduit)

nbrs = NearestNeighbors(n_neighbors=6, algorithm='ball_tree').fit(X_ACP)
dist_KNN_ACP, ind_KNN_ACP = nbrs.kneighbors(X_ACP)

In [ ]:
#Algorithme KNN plus proche voisin après T_SNE
from sklearn.manifold import TSNE
tsne = TSNE()
X_TSNE = tsne.fit_transform(df_CentreReduit)

nbrs = NearestNeighbors(n_neighbors=6, algorithm='ball_tree').fit(X_TSNE)
dist_KNN_TSNE, ind_KNN_TSNE = nbrs.kneighbors(X_TSNE)

In [ ]:
#K-MEANS DETECTION DU NOMBRE DE CLUSTERS POUR TSNE
sse = {}
silhouette_avg = {}
plusPetitCluster = {}
for k in tqdm(range(25, 60)):
    kmeans = KMeans(n_clusters=k, max_iter=1000).fit(X_TSNE)
    sse[k] = kmeans.inertia_ # Inertia: Sum of distances of samples to their closest cluster center
    silhouette_avg[k] = silhouette_score(X_TSNE, kmeans.labels_)
    plusPetitCluster[k] = collections.Counter(kmeans.labels_).most_common()[len(collections.Counter(kmeans.labels_))-1][1]
    
plt.plot(list(sse.keys()),list(sse.values()))
plt.xlabel('Nbre de cluster')
plt.ylabel('Inertie')
plt.title('Intertie suivant le nombre de Cluster')
plt.show()

plt.plot(list(silhouette_avg.keys()),list(silhouette_avg.values()))
plt.xlabel('Nbre de cluster')
plt.ylabel('Silhouette')
plt.title('Silhouette suivant le nombre de Cluster')
plt.show()

plt.plot(list(plusPetitCluster.keys()),list(plusPetitCluster.values()))
plt.xlabel('Nbre de Cluster')
plt.ylabel('Taille du Plus petit Cluster')
plt.title('Evolution de la taille du plus petit Cluster')
plt.show()

In [ ]:
#K-MEANS DETECTION DU NOMBRE DE CLUSTERS POUR DONNEES BRUTES
sse = {}
silhouette_avg = {}
plusPetitCluster = {}
for k in tqdm(range(15, 90)):
    kmeans = KMeans(n_clusters=k, max_iter=1000).fit(df_CentreReduit)
    sse[k] = kmeans.inertia_ # Inertia: Sum of distances of samples to their closest cluster center
    silhouette_avg[k] = silhouette_score(df_CentreReduit, kmeans.labels_)
    plusPetitCluster[k] = collections.Counter(kmeans.labels_).most_common()[len(collections.Counter(kmeans.labels_))-1][1]
    
plt.plot(list(sse.keys()),list(sse.values()))
plt.xlabel('Nbre de cluster')
plt.ylabel('Inertie')
plt.title('Intertie suivant le nombre de Cluster')
plt.show()

plt.plot(list(silhouette_avg.keys()),list(silhouette_avg.values()))
plt.xlabel('Nbre de cluster')
plt.ylabel('Silhouette')
plt.title('Silhouette suivant le nombre de Cluster')
plt.show()

plt.plot(list(plusPetitCluster.keys()),list(plusPetitCluster.values()))
plt.xlabel('Nbre de Cluster')
plt.ylabel('Taille du Plus petit Cluster')
plt.title('Evolution de la taille du plus petit Cluster')
plt.show()

In [ ]:
#K-MEANS DETECTION DU NOMBRE DE CLUSTERS POUR DONNEES ACP
sse = {}
silhouette_avg = {}
plusPetitCluster = {}
for k in tqdm(range(2, 20)):
    kmeans = KMeans(n_clusters=k, max_iter=1000).fit(X_ACP)
    sse[k] = kmeans.inertia_ # Inertia: Sum of distances of samples to their closest cluster center
    silhouette_avg[k] = silhouette_score(X_ACP, kmeans.labels_)
    plusPetitCluster[k] = collections.Counter(kmeans.labels_).most_common()[len(collections.Counter(kmeans.labels_))-1][1]
    
plt.plot(list(sse.keys()),list(sse.values()))
plt.xlabel('Nbre de cluster')
plt.ylabel('Inertie')
plt.title('Intertie suivant le nombre de Cluster')
plt.show()

plt.plot(list(silhouette_avg.keys()),list(silhouette_avg.values()))
plt.xlabel('Nbre de cluster')
plt.ylabel('Silhouette')
plt.title('Silhouette suivant le nombre de Cluster')
plt.show()

plt.plot(list(plusPetitCluster.keys()),list(plusPetitCluster.values()))
plt.xlabel('Nbre de Cluster')
plt.ylabel('Taille du Plus petit Cluster')
plt.title('Evolution de la taille du plus petit Cluster')
plt.show()

In [ ]:
#Kmeans avec 25 clusters sur Données Brute
kmeans = KMeans(n_clusters=80, max_iter=1000).fit(df_CentreReduit)
kmeans_labels_Brute = kmeans.labels_
print(silhouette_score(df_CentreReduit, kmeans_labels_Brute))

In [ ]:
#Kmeans avec 25 clusters sur ACP
kmeans = KMeans(n_clusters=8, max_iter=1000).fit(X_ACP)
kmeans_labels_ACP = kmeans.labels_
print(silhouette_score(X_ACP, kmeans_labels_ACP))

In [ ]:
#Kmeans avec 25 clusters sur TSNE
kmeans = KMeans(n_clusters=48, max_iter=1000).fit(X_TSNE)
kmeans_labels_TSNE = kmeans.labels_
print(silhouette_score(X_TSNE, kmeans_labels_TSNE))

In [ ]:
#GOWER
from sklearn.neighbors import DistanceMetric
from sklearn.preprocessing import MinMaxScaler
#Calcul des distances en prenant en compte les données quantitatives des qualitatives

#indicateurs numériques
indNum = ['num_critic_for_reviews','duration','director_facebook_likes','num_voted_users','cast_total_facebook_likes','budget',
          'title_year','imdb_score','movie_facebook_likes','actor_1_facebook_likes','num_user_for_reviews','actor_2_facebook_likes']

df_Numerique = df_full_cat.loc[:,indNum]
df_Categorie = df_full_cat.drop(indNum,axis=1)
df_Categorie = df_Categorie.drop(["movie_title",'country'],axis=1)

#Calcul de la distance sur les données numériques
DistanceMetrique = DistanceMetric.get_metric('manhattan').pairwise(df_Numerique)
# Centrage et Réductions
#std_scale = preprocessing.StandardScaler().fit(DistanceMetrique)
#DistanceMetrique_CentreReduit = std_scale.transform(DistanceMetrique)

scaler = MinMaxScaler()
scaler.fit(DistanceMetrique)
DistanceMetrique_CentreReduit = scaler.transform(DistanceMetrique)

#Calcul de la distance sur les données catégorielles
DistanceCategoriel = DistanceMetric.get_metric('dice').pairwise(df_Categorie)

#Calcul de GOWER
#Distance catégoriel + Distance métrique divisé par le nombre de variable
DistanceTotal  = (DistanceCategoriel + DistanceMetrique_CentreReduit) / df_Categorie.shape[1] + df_Numerique.shape[1]

In [ ]:
#Ecriture du fichier à exploiter au niveau de l'API
dfForApi = pd.DataFrame(ind_KNN_TSNE)
dfForApi.insert(0, "nomFilm", df['movie_title'], allow_duplicates=False)
dfForApi.to_csv('KNN_BRUTE.csv', sep = '\t', encoding='UTF-8')